# Metaclass
- Meta classes allow us to intercept and augment class creation

## Usage
- Add decoration to all methods of classes
- Register all classes in use to an API
- User Interface logic
- Create or extend classes from simplified specifactions in text files

## Metaclass VS Class Decorator
- Class decorator is designed to manage instance
- Metaclass are designed to augment class construction
- The main differnece difference between class decorators and metaclass is their place in the timing of class creation

## Metaclass VS Helper Function
- The advantage of metaclass
    - explicit structure
    - consistentcy
    - avoid code redundancy
    
### Example
Add a extra function to a class

In [1]:
# Function to add
def extra(self, arg):
    print("[From extra]: ", arg)

In [2]:
# Version 1: Helper Function    
def extras(cls):
    cls.extra = extra
    
class C1:
    pass
extras(C1)

class C2:
    pass
extras(C2)

In [3]:
# Version 2: Metaclass

class Extras(type):
    def __init__(cls, clsname, superclasses, attributedict):
        cls.extra = extra
        
class C1(metaclass=Extras):
    pass

class C2(metaclass=Extras):
    pass

## Class Statement Protocol

- **type** is a class that generates user-defined classes
- Metaclasses are subclasses of the **type** class
- Class objects are instnces of the **type** class, or a subclass
- Instance objects are generated form a class

### Behind Class Statement
```python
class = type(classname, superclasses, attributeddict)
```

**type** defines a **`__call__`** and runs the following methods
```python
type.__new__(typeclass, classname, superclasses, attributedict)
type.__init__(cls, classname, superclasses, attributedict)
```


The following two cells are equivalent

In [4]:
class A:
    pass

class B(A):
    data = 1
    def meth(self, arg):
        return self.data + arg
    
b = B()
b.meth(2)

3

In [5]:
B = type('B', (A, ), {'data': 1, 'meth': (lambda x, y: x.data + y)})

b = B()
b.meth(2)

3

## The Basic Idea of Metaclasses
Thus, to control the way classes are created and augment their behavior.  
We have to specify that a user-defined class be created from a user-defined metaclass instead of the normal **type** class.

## Declaration of Metaclasses
- Superclasses must be listed before the metaclass
- There is no simple protability between version
    - Python3 ignores the Python2 `__metaclass__`
    - Python2 regards the metaclass keyword as syntax error

### Python3
```python
class A(metaclass=Meta):
    pass
    
class B(C, metaclass=Meta):
    pass
```

### Python2
```python
class A(object):
    __metaclass = Meta
    
class B(C, object):
    __metaclass__ = Meta
```

Although classes derived from **object** explicityly is not a must in Python2, the `__metaclass__` declaration makes the resulting class new-style.  
Thus, it's suggested to explicitly declared

### Behind The Declaration
Replace the *type* with *metaclass* when creating class

```python
class = Meta(clsname, superclasses, attributedict)
```

Because metaclass is a subclass of type, the **`__call__`** is delegated
```python
Meta.__new__(Meta, clsname, superclasses, attributedict)
Meta.__init__(cls, classname, superclasses, attributedict)
```

## Implement Metaclasses

### A Baic Metaclass

In [6]:
class Meta(type):
    def __new__(meta, classname, supers, classdict):
        print('Meta New: ', meta, classname, supers, classdict, sep='\n\t...')
        return type.__new__(meta, classname, supers, classdict)
    
    def __init__(cls, classname, supers, classdict):
        print('Meta New: ', cls, classname, supers, classdict, sep='\n\t...')
        
class A:
    pass

print('---Create class B---')
class B(A, metaclass=Meta):
    data = 1
    def __init__(self):
        print('Create instance of B')
    
    def meth(self, arg):
        return self.data + arg
    
print('---Create instance b---')
b = B()

---Create class B---
Meta New: 
	...<class '__main__.Meta'>
	...B
	...(<class '__main__.A'>,)
	...{'__module__': '__main__', '__qualname__': 'B', 'data': 1, '__init__': <function B.__init__ at 0x1044b0d90>, 'meth': <function B.meth at 0x1044b0d08>}
Meta New: 
	...<class '__main__.B'>
	...B
	...(<class '__main__.A'>,)
	...{'__module__': '__main__', '__qualname__': 'B', 'data': 1, '__init__': <function B.__init__ at 0x1044b0d90>, 'meth': <function B.meth at 0x1044b0d08>}
---Create instance b---
Create instance of B


### Other Metaclass Techniques
Metaclasses need not really be classes, it can be any callable object that accepts the arguments passed and returns an object compatible with the intended class

In [7]:
def MetaFunc(clsname, supers, clsdict):
    print('MetaFunc: ', clsname, supers, clsdict, sep='\n\t...')
    return type(clsname, supers, clsdict)

        
class A:
    pass

print('---Create class B---')
class B(A, metaclass=MetaFunc):
    data = 1
    def __init__(self):
        print('Create instance of B')
    
    def meth(self, arg):
        return self.data + arg
    
print('---Create instance b---')
b = B()

---Create class B---
MetaFunc: 
	...B
	...(<class '__main__.A'>,)
	...{'__module__': '__main__', '__qualname__': 'B', 'data': 1, '__init__': <function B.__init__ at 0x1044b0598>, 'meth': <function B.meth at 0x1044b01e0>}
---Create instance b---
Create instance of B


## Inheritance an Instance
- Metaclass Declrations are inherited by subclasses
- Metaclass attributes are not inherited by class instance
- Metaclass attribute are aquired by classes

In [8]:
# Definition

class Meta(type):
    def __new__(meta, clsname, supers, clsdict):
        print('Meta: ', clsname)
        return type.__new__(meta, clsname, supers, clsdict)
    
    def toast(self):
        return 'toast'
    
class Super(metaclass=Meta):
    def spam(self):
        return 'spam'
    
class Sub(Super):
    def eggs(self):
        return 'eggs'

Meta:  Super
Meta:  Sub


In [9]:
# Instance

x = Sub()
x.eggs()
x.spam()
x.toast()

AttributeError: 'Sub' object has no attribute 'toast'

In [10]:
# Class
x = Sub()
Sub.eggs(x)
Sub.spam(x)
Sub.toast()

'toast'

## Metaclass Methods VS Class Method
- Much the same, but
    - Metaclasses methods are not accessible excpet through the class
    - Metaclasses do not need **classmethod** declarator